# Основы прикладной математики и информатики

# Экзамен | Демоверсия | Python 

[Источник данных - Kaggle](https://www.kaggle.com/datasets/narayan63/netflix-popular-movies-dataset)

Данные содержат следующие признаки:

* `title` - Название фильма
* `year` - Год выпуска фильма
* `certificate` - Возрастное ограничение
* `duration` - Длительность
* `genre` - Жанр фильма
* `rating` - Рейтиг фильма
* `description` - Описание
* `stars` - Актеры
* `votes` - Количество оценок фильма

Пользоваться можно всем, кроме помощи одногруппников. Время на экрамен -- 2 пары.  

In [174]:
import pandas as pd
import re
import plotly.graph_objects as go
from bs4 import BeautifulSoup

data = pd.read_csv('https://raw.githubusercontent.com/aaparshina/DDC_22-23_basics_python/main/data/movies.csv')
data.head()

,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413"


## Часть 1. Работа с данными - фильтрация, сортировка, группировка, создание новых признаков

**1. *(0.25 балла)* Сколько всего уникальных фильмов в данных? Сколько всего фильмов с рейтингом выше 7.5?**

In [177]:
len(data['title'].unique())

6261

In [178]:
len(data[data['rating'] > 7.5]['title'].unique())

1283

**2. *(0.25 балла)* Выберите фильмы с самым часто встречающимся возрастным ограничением, отсортируйте полученные данные по рейтингу. У какого фильма будет минимальный/максимальный рейтинг?**

In [179]:
data['certificate'].value_counts()

TV-MA        2197
TV-14        1081
R             466
TV-PG         464
Not Rated     458
PG-13         276
TV-Y7         266
PG            154
TV-G          114
TV-Y          109
Unrated        63
G              43
Approved       25
TV-Y7-FV       25
Passed          6
NC-17           4
MA-17           1
12              1
M               1
Name: certificate, dtype: int64

In [180]:
data[data['certificate'] == 'TV-MA'].sort_values(by = 'rating')['title'].head(1)

3280    The Hype House
Name: title, dtype: object

In [181]:
data[data['certificate'] == 'TV-MA'].sort_values(by = 'rating', ascending = False)['title'].head(1)

7321    BoJack Horseman
Name: title, dtype: object

**3. *(0.25 балла)* Сколько уникальных драматических (Drama) фильмов есть в данных?**

In [212]:
genre = data.dropna(subset = 'genre')
len(genre[genre['genre'].str.contains("Drama")]['title'].unique())

2671

**4. *(0.5 балла)* Создайте столбец с продолжительностью фильма, выраженной числовым значением. Выберите только те фильмы, которые длились больше пяти часов (300 минут). Сколько таких фильмов?**

In [183]:
data['dur_new'] = data['duration'].apply(lambda x: int(x.split()[0]))
data.head()

,title,year,certificate,duration,genre,rating,description,stars,votes,dur_new
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031",30
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885",58
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384",46
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773",356
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413",24


In [184]:
len(data[data['dur_new'] > 300]['title'].unique())

55

**5. *(0.25 балла)* Отсортируйте данные по длительности фильма. Выведите описание самого длинного фильма.**

In [187]:
data.sort_values(by = 'dur_new', ascending = False).loc[896]['description']

"A comprehensive history of the United States' involvement in the bitterly divisive armed conflict in Southeast Asia."

**6. *(0.5 балла)* Опасно: регулярные выражения!**

**Создайте столбец с годом фильма (используйте для преобразования столбец year; если там два года, то выбирайте первый). Используя новый столбец выберите только те фильмы, которые были выпущены с 2016 по 2020 года. Сгруппируйте полученные данные по возрастному ограничению и году фильма, посчитайте средний рейтинг. Выведите результат в виде таблицы.**

In [188]:
data['year_start'] = data['year'].apply(lambda x: int(re.findall(r'\d{4}', x)[0]))
data.head()

,title,year,certificate,duration,genre,rating,description,stars,votes,dur_new,year_start
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031",30,2018
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885",58,2016
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384",46,2015
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773",356,2022
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413",24,2022


In [189]:
data_2015 = data[(data['year_start'] > 2015) & (data['year_start'] <= 2020)]
data_2015.groupby(['certificate', 'year_start'])['rating'].mean().unstack()

year_start,2016,2017,2018,2019,2020
certificate,,,,,
Approved,NaN,NaN,7.800000,NaN,NaN
G,6.600000,6.433333,NaN,6.800000,5.900000
M,NaN,NaN,NaN,5.800000,NaN
Not Rated,5.679592,5.840000,5.934783,5.673077,6.855556
PG,6.940000,6.181818,6.177778,6.400000,6.175000
PG-13,6.286364,6.030000,6.023810,6.193750,6.436000
R,6.118421,5.990323,6.353125,6.097222,6.010811
TV-14,7.434021,6.888350,7.165909,6.919298,6.834211
TV-G,6.614286,6.850000,6.522222,7.716667,7.123333


## Часть 2. Меры центральной тенденции и меры разброса, корреляция

**7. *(0.25 балла)* Какова средняя продолжительность фильма в 2022 году?**

In [190]:
data[data['year_start'] == 2022]['dur_new'].mean()

84.37407407407407

**8. *(0.5 балла)* Преобразуйте столбец votes (можно на его основе создать новый), чтобы с ним можно было проводить вычисления (уберите запятую, преобразуйте в целое число). Найдите выбросы по новой переменной, используя межквартильный размах. Сколько фильмов останется, если убрать (отфильтровать данные) выбросы?**

In [191]:
data['votes_2'] = data['votes'].apply(lambda x: int(x.replace(',', '')))
data.head()

,title,year,certificate,duration,genre,rating,description,stars,votes,dur_new,year_start,votes_2
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031",30,2018,177031
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885",58,2016,199885
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384",46,2015,501384
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773",356,2022,9773
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413",24,2022,15413


In [209]:
q1 = data['votes_2'].quantile(0.25)
q3 = data['votes_2'].quantile(0.75)

iqr = q3-q1

bottom = q1 - 1.5 * iqr
top = q3 + 1.5 * iqr

len(data[data['votes_2'] <= top]['title'].unique())

5178

**9. *(0.25 балла)* Посчитайте среднее и медиану по рейтингу до и после удаления выбросов (предыдущий пункт). Как изменились меры?**

In [210]:
no_outliners = data[data['votes_2'] <= top]

print(data['rating'].mean(), data['rating'].median())
print(no_outliners['rating'].mean(), no_outliners['rating'].median())

6.708567708333334 6.8
6.640713178294566 6.8


**10. *(0.25 балла)* Укажите фильм с максимальным количеством голсов.**

In [211]:
data[data['votes_2'] == data['votes_2'].max()]

,title,year,certificate,duration,genre,rating,description,stars,votes,dur_new,year_start,votes_2
7645,The Lord of the Rings: The Fellowship of the Ring,(2001),PG-13,178 min,"Action, Adventure, Drama",8.8,A meek Hobbit from the Shire and eight compani...,"['Peter Jackson', '| ', ' Stars:', 'Elijah ...","1,844,075",178,2001,1844075


**11. *(0.5 балла)* Постройте корреляционную матрицу для рейтига, количества голосов и длительности фильма. Прокомментируйте каждый коэффициент корреляции (укажите тип взаимосвязи и силу).**

In [200]:
data[['rating', 'votes_2', 'dur_new']].corr()
# rating x votes_2 -- прямая, слабая
# rating x dur_new -- обратная, слабая
# votes_2 x dur_new -- прямая, слабая

,rating,votes_2,dur_new
rating,1.000000,0.161555,-0.166247
votes_2,0.161555,1.000000,0.107793
dur_new,-0.166247,0.107793,1.000000


**12. *(0.25 балла)* Сколько в данных фильмов с рейтингом выше среднего?**

In [204]:
len(data[data['rating'] > data['rating'].mean()]['title'].unique())

2865

**13. *(0.5 балла)* Выберите все фильмы, в описании которых есть слово love. У какого фильма наибольший рейтинг? У какого фильма наибольшее количество голосов? В каком году было выпущено большинство из них?**

In [216]:
love = data[data['description'].str.contains('love')]
love.head()

,title,year,certificate,duration,genre,rating,description,stars,votes,dur_new,year_start,votes_2
56,The Vampire Diaries,(2009–2017),TV-14,43 min,"Drama, Fantasy, Horror",7.7,"The lives, loves, dangers and disasters in the...","['Nina Dobrev, ', 'Paul Wesley, ', 'Ian Somerh...","318,383",43,2009,318383
61,Diario de un Gigoló,(2022),TV-MA,50 min,Drama,6.3,A gigolo's life begins to unravel when he beco...,"['Jesús Castro, ', 'Victoria White, ', 'Fabiol...",538,50,2022,538
63,Heartbreak High,(2022– ),TV-MA,50 min,"Drama, Romance",7.6,A fresh look at Hartley High over 20 years on....,"['Ayesha Madon, ', 'James Majoos, ', 'Chloe Ha...","1,502",50,2022,1502
88,365 Days,(2020),TV-MA,114 min,"Drama, Romance",3.3,Massimo is a member of the Sicilian Mafia fami...,"['Barbara Bialowas, ', 'Tomasz Mandes', '| ', ...","89,071",114,2020,89071
90,Titanic,(1997),PG-13,194 min,"Drama, Romance",7.9,A seventeen-year-old aristocrat falls in love ...,"['James Cameron', '| ', ' Stars:', 'Leonard...","1,158,746",194,1997,1158746


In [217]:
love[love['rating'] == max(love['rating'])]

,title,year,certificate,duration,genre,rating,description,stars,votes,dur_new,year_start,votes_2
6530,Dexter,(2006–2013),TV-MA,56 min,"Crime, Drama, Mystery",9.6,"In the Season One finale, Dexter follows the c...","['Michael Cuesta', '| ', ' Stars:', 'Michae...","10,604",56,2006,10604


In [218]:
love[love['votes_2'] == max(love['votes_2'])]

,title,year,certificate,duration,genre,rating,description,stars,votes,dur_new,year_start,votes_2
90,Titanic,(1997),PG-13,194 min,"Drama, Romance",7.9,A seventeen-year-old aristocrat falls in love ...,"['James Cameron', '| ', ' Stars:', 'Leonard...","1,158,746",194,1997,1158746


In [219]:
love['year_start'].mode()

0    2020
Name: year_start, dtype: int64

## Часть 3. Визуализация

**13. *(1.5 балла)* Постройте линейный график, где по оси Х -- года, по оси Y -- количество фильмов за каждый год.**

1. Сгруппируйте данные по годам, по столбцу title посмотрите на .count(). Сделайте из этого датафрейм (вспоминаем функцию DataFrame()).
2. Используйте функцию Scatter(), где в качестве х укажите индексы нового датафрейма, а по у -- его единственный столбец title.
3. Поменяйте цвет линии. Добавьте загловок, подпись осей.  

In [238]:
to_plot = pd.DataFrame(data.groupby('year_start')['title'].count())
to_plot.head()

,title
year_start,
1932,2
1933,1
1934,1
1935,1
1938,1


In [239]:
trace = go.Scatter(  
    x=to_plot.index,
    y=to_plot.title,
    marker={'color': 'grey'}
)

our_data = [trace] 

our_layout = go.Layout(
    title="Количество фильмов по годам", 
    xaxis={'title':'Год'}, 
    yaxis={'title':'Фильмы'})


fig = go.Figure(data=our_data, layout=our_layout)

fig.show()

## Часть 4. Регулярные выражения


**14. *(2 балла)* Дан текст. Используйте регулярные выражения и вытащите из него следующую информацию:**

1. Всех отправителей
2. Все даты
3. Все время (в формате hh:mm:ss)
4. ID сообщения (считаем, что ID -- это первые цифры до точки, например, вот здесь `<200801051412.m05ECIaH010327@nakamura.uits.iupui.edu>` ID -- `200801051412`

In [249]:
emails = \
'To: source@collab.sakaiproject.org\n\
From: stephen.marquard@uct.ac.za\n\
Message-ID: <200801051412.m05ECIaH010327@nakamura.uits.iupui.edu>\n\
Date: Sat, 5 Jan 2008 09:12:18\n\
To: source@collab.sakaiproject.org\n\
From: louis@media.berkeley.edu\n\
Message-ID: <200801042308.m04N8v6O008125@nakamura.uits.iupui.edu>\n\
Date: Fri, 4 Jan 2008 18:08:57\n\
To: source@collab.sakaiproject.org\n\
From: zqian@umich.edu\n\
Message-ID: <200801042109.m04L92hb007923@nakamura.uits.iupui.edu>\n\
Date: Fri, 14 Jan 2008 16:09:02\n\
To: source@collab.sakaiproject.org\n\
From: rjlowe@iupui.edu\n\
Message-ID: <200801042044.m04Kiem3007881@nakamura.uits.iupui.edu>\n\
Date: Fri, 4 Jan 2008 15:44:40'
print(emails)

To: source@collab.sakaiproject.org
From: stephen.marquard@uct.ac.za
Message-ID: <200801051412.m05ECIaH010327@nakamura.uits.iupui.edu>
Date: Sat, 5 Jan 2008 09:12:18
To: source@collab.sakaiproject.org
From: louis@media.berkeley.edu
Message-ID: <200801042308.m04N8v6O008125@nakamura.uits.iupui.edu>
Date: Fri, 4 Jan 2008 18:08:57
To: source@collab.sakaiproject.org
From: zqian@umich.edu
Message-ID: <200801042109.m04L92hb007923@nakamura.uits.iupui.edu>
Date: Fri, 14 Jan 2008 16:09:02
To: source@collab.sakaiproject.org
From: rjlowe@iupui.edu
Message-ID: <200801042044.m04Kiem3007881@nakamura.uits.iupui.edu>
Date: Fri, 4 Jan 2008 15:44:40


In [251]:
# отправители
re.findall(r'From: (.+)\n', emails)

['stephen.marquard@uct.ac.za',
 'louis@media.berkeley.edu',
 'zqian@umich.edu',
 'rjlowe@iupui.edu']

In [252]:
# даты
re.findall(r'\d{1,2} \w{3} \d{4}', emails)

['5 Jan 2008', '4 Jan 2008', '14 Jan 2008', '4 Jan 2008']

In [253]:
# время
re.findall(r'\d{2}:\d{2}:\d{2}', emails)

['09:12:18', '18:08:57', '16:09:02', '15:44:40']

In [255]:
# ID
re.findall(r'<(\d+)\.', emails)

['200801051412', '200801042308', '200801042109', '200801042044']

## Часть 5. Работа с html

**15. *(2 балла)* Дан кусочек кода страницы новостного сайта. Вытащите из него следующую информацию:**

1. Ссылки на новости
2. Заголовки новостей
3. Время публикации новости

**Результаты соберите в таблицу. Назовите колонки. Сохраните получившиеся данные.**

In [278]:
soup = BeautifulSoup(
    '<li class="news-listing__item news-listing__item_hot"><a href="https://www.mk.ru/politics/2022/10/19/sovfed-utverdil-ukaz-putina-o-voennom-polozhenii-v-novykh-regionakh.html" class="news-listing__item-link"><span class="news-listing__item-time">17:32</span><h3 class="news-listing__item-title ">Совфед утвердил указ Путина о военном положении в новых регионах</h3></a></li>\
    <li class="news-listing__item "><a href="https://www.mk.ru/social/2022/10/19/v-rostovskoy-oblasti-zayavili-chto-ne-vvodyat-zapret-na-vezd-i-vyezd.html" class="news-listing__item-link"><span class="news-listing__item-time">17:26</span><h3 class="news-listing__item-title ">В Ростовской области заявили, что не вводят запрет на въезд и выезд</h3></a></li>\
    <li class="news-listing__item "><a href="https://www.mk.ru/social/2022/10/19/priluchnyy-vystupil-v-sude-po-delu-ob-izbienii-v-nochnom-klube.html" class="news-listing__item-link"><span class="news-listing__item-time">17:25</span><h3 class="news-listing__item-title ">Прилучный выступил в суде по делу об избиении в ночном клубе</h3></a></li>\
    <li class="news-listing__item "><a href="https://www.mk.ru/politics/2022/10/19/sozdatel-chvk-vagner-prigozhin-podtverdil-nachalo-sozdaniya-narodnogo-opolcheniya.html" class="news-listing__item-link"><span class="news-listing__item-time">17:22</span><h3 class="news-listing__item-title ">Создатель ЧВК «Вагнер» Пригожин подтвердил начало создания народного ополчения</h3></a></li>'
)
print(soup)

<html><body><li class="news-listing__item news-listing__item_hot"><a class="news-listing__item-link" href="https://www.mk.ru/politics/2022/10/19/sovfed-utverdil-ukaz-putina-o-voennom-polozhenii-v-novykh-regionakh.html"><span class="news-listing__item-time">17:32</span><h3 class="news-listing__item-title">Совфед утвердил указ Путина о военном положении в новых регионах</h3></a></li> <li class="news-listing__item"><a class="news-listing__item-link" href="https://www.mk.ru/social/2022/10/19/v-rostovskoy-oblasti-zayavili-chto-ne-vvodyat-zapret-na-vezd-i-vyezd.html"><span class="news-listing__item-time">17:26</span><h3 class="news-listing__item-title">В Ростовской области заявили, что не вводят запрет на въезд и выезд</h3></a></li> <li class="news-listing__item"><a class="news-listing__item-link" href="https://www.mk.ru/social/2022/10/19/priluchnyy-vystupil-v-sude-po-delu-ob-izbienii-v-nochnom-klube.html"><span class="news-listing__item-time">17:25</span><h3 class="news-listing__item-title"

In [287]:
# ссылки
urls = [i.get('href') for i in soup.find_all('a')]
urls

['https://www.mk.ru/politics/2022/10/19/sovfed-utverdil-ukaz-putina-o-voennom-polozhenii-v-novykh-regionakh.html',
 'https://www.mk.ru/social/2022/10/19/v-rostovskoy-oblasti-zayavili-chto-ne-vvodyat-zapret-na-vezd-i-vyezd.html',
 'https://www.mk.ru/social/2022/10/19/priluchnyy-vystupil-v-sude-po-delu-ob-izbienii-v-nochnom-klube.html',
 'https://www.mk.ru/politics/2022/10/19/sozdatel-chvk-vagner-prigozhin-podtverdil-nachalo-sozdaniya-narodnogo-opolcheniya.html']

In [288]:
# заголовки новостей
title = [i.text for i in soup.find_all('h3')]
title

['Совфед утвердил указ Путина о военном положении в новых регионах',
 'В Ростовской области заявили, что не вводят запрет на въезд и выезд',
 'Прилучный выступил в суде по делу об избиении в ночном клубе',
 'Создатель ЧВК «Вагнер» Пригожин подтвердил начало создания народного ополчения']

In [289]:
# время
time = [i.text for i in soup.find_all('span')]
time

['17:32', '17:26', '17:25', '17:22']

In [293]:
news = pd.DataFrame([urls, title, time]).transpose()
news.columns = ['Ссылка', 'Заголовок', 'Время']
news.to_csv('news.csv')